In [ ]:
import numpy as np
import spectral as sp
import matplotlib.pyplot as plt
import cv2
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from functions import *

img12h = sp.open_image(getImage("12h"))
img3h = sp.open_image(getImage("3h"))

n, m, bands = img12h.shape

bruiseCoords12h = np.array(([1358, 5100], [231, 5082], [240,3873]))
bruiseCoords3h = np.array(([1435, 5394], [320, 5344], [327, 4157]))
bruiseKernel = 25

# put all bruised regions in one list
bruisedRegion = []
for i in range(bruiseCoords12h.shape[0]):
    bruisedRegion.append(getRegion(img3h, bruiseCoords3h[i,0], bruiseCoords3h[i,1], bruiseKernel))
bruisedRegion = np.array(bruisedRegion) # make array from list

# make 3D array instead of 4D
bruise = np.zeros((bruisedRegion.shape[1]*3, bruisedRegion.shape[2], bands))

# prøvde å gjøre innlesingen litt mer fancy, men den går out of bounds pga (i+1)
#for i in range(bruisedRegion.shape[1]):
#    bruise[i*bruisedRegion.shape[1]:#####denne går out of bounds(i+1)######*bruisedRegion.shape[1]] = bruisedRegion[i]

# place all regions in same 3D array
bruise[:bruisedRegion.shape[1]] = bruisedRegion[0]
bruise[bruisedRegion.shape[1]: 2*bruisedRegion.shape[1]] = bruisedRegion[1]
bruise[2*bruisedRegion.shape[1]:] = bruisedRegion[2]

# reshape array for svm
bruise = bruise.reshape((bruise.shape[0]*bruise.shape[1], bands))

healthyCoords12h = np.array(([1184, 5041], [246, 4910], [450, 3888]))
healthyCoords3h = np.array(([1231, 5202], [550, 5175], [580,4088]))
healthyKernel = 19

# put all healthy regions in one list
healthyRegion = []
for i in range(healthyCoords12h.shape[0]):
    healthyRegion.append(getRegion(img3h, healthyCoords3h[i,0], healthyCoords3h[i,1], healthyKernel))
healthyRegion = np.array(healthyRegion) # make array from list

# make 3D array instead of 4D
healthy = np.zeros((healthyRegion.shape[1]*3, healthyRegion.shape[2], bands))

# place all regions in same 3D array
healthy[:healthyRegion.shape[1]] = healthyRegion[0]
healthy[healthyRegion.shape[1]: 2*healthyRegion.shape[1]] = healthyRegion[1]
healthy[2*healthyRegion.shape[1]:] = healthyRegion[2]

# reshape array for svm
healthy = healthy.reshape((healthy.shape[0]*healthy.shape[1], bands))

In [ ]:
X_train = np.zeros((bruiseKernel**2 * 2 + healthyKernel**2 * 2, bands))
X_train[:bruiseKernel**2 * 2] = bruise[:bruiseKernel**2 * 2]  # adding bruised train data
X_train[bruiseKernel**2 * 2:] = healthy[:healthyKernel**2 *2] # adding healthy train data

X_test = np.zeros((bruiseKernel**2 + healthyKernel**2, bands))
X_test[:bruiseKernel**2] = bruise[bruiseKernel**2 * 2:] # adding bruised test data
X_test[bruiseKernel**2:] = healthy[healthyKernel**2 *2:] # adding healthy train data

Y_train = np.ones(X_train.shape[0])
Y_train[bruiseKernel**2*2:] = 0

Y_test = np.ones(X_test.shape[0])
Y_test[bruiseKernel**2:] = 0


In [ ]:
SVM = svm.SVC()
SVM.fit(X_train, Y_train)

y_pred = SVM.predict(X_test)

correct = 0
total = len(y_pred)

for i in range(len(y_pred)):
    if y_pred[i] == Y_test[i]:
        correct += 1

print(correct/total)
#print(SVM.score(Y_test, y_pred))

In [ ]:
KNN = KNeighborsClassifier()
KNN.fit(X_train, Y_train)

y_pred_KNN = KNN.predict(X_test)

correct = 0
total = len(y_pred_KNN)

for i in range(len(y_pred_KNN)):
    if y_pred_KNN[i] == Y_test[i]:
        correct += 1

print(correct/total)
#print(KNN.score(Y_test, y_pred_KNN))

In [ ]:
# parameters for inspecting bruised area
kernel = 401
x_coord = 330
y_coord = 4150

X_test = getRegion(img3h, x_coord, y_coord, kernel)
X_test = X_test.reshape((kernel**2, 186))


SVM = svm.SVC()
SVM.fit(X_train, Y_train)

y_pred = SVM.predict(X_test)
y_pred = y_pred.reshape((kernel,kernel))
plt.imshow(y_pred, cmap='gray')